# Verificación de Errores

En este notebook vamos a verificar que todas las imágenes tengan el shape correcto (250, 250, 3), que el rango de los valores se encuentre entre 0 y 255 y que no contenga valores nulos. 

In [2]:
import numpy as np
import cv2
import pandas as pd
import os
from tqdm import tqdm

In [4]:
def get_labels_and_paths(input_folder):
   return sorted([(dp.split("/")[1],os.path.join(dp, f)) for dp, dn, filenames in os.walk('input') for f in filenames if os.path.splitext(f)[1] == '.jpg'])

def img_to_numpy(path): 
    img = cv2.imread(filename=path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return img

def shape_mal(img_path:str, img_shape=(250,250,3))->bool:
    return cv2.imread(filename=img_path).shape!=img_shape

def rango_mal(img_path:str)->bool:
    flattened = cv2.imread(filename=img_path).flatten()
    return np.sum((flattened>=0) & (flattened<=255)) != len(flattened)

def contains_nulls(img_path:str)->bool:
    img = cv2.imread(filename=img_path)
    return np.sum(np.isnan(img)) > 0

def chequeo_completo(img_path:str)->bool:
    return np.array([shape_mal(img_path), rango_mal(img_path), contains_nulls(img_path)])

def retorno_error(booleanos)->bool:
    return np.array(['shape mal','rango mal','contiene nulos'])[booleanos]

def verificar_imagenes(all_paths:list)->dict:
    errores = {}
    for img_path in tqdm(all_paths):
        booleanos = chequeo_completo(img_path)
        if any(booleanos): 
            errores[img_path.split("/")[-1]] = retorno_error(booleanos=booleanos)
    if errores != {}:
        return errores
    else:
        print("Ninguna imagen contiene errores")
        return None

In [5]:
# Verificacion de errores

INPUT_FOLDER = "input/"
labels_and_paths = get_labels_and_paths(input_folder=INPUT_FOLDER)
input_paths = [x[1] for x in labels_and_paths]

verificar_imagenes(all_paths=input_paths)

100%|██████████| 75000/75000 [01:55<00:00, 651.68it/s]

Ninguna imagen contiene errores
